# Notebook to check MELD_BIDS folder


In [1]:
import os
import glob
import pandas as pd
import numpy as np
import json
import shutil

In [79]:
# initialise 

site_code = 'H102'
site_folder = f'/home/mathilde/Documents/RDS/MELD_FE/DATA/MELD_{site_code}'


### 1. Check folder MELD_BIDS

In [80]:
def harmonise_meld_name(name):
    split = name.split('_')  
    # exclude specific characters
    harmo_name= ''.join(split)
    
    return harmo_name


In [81]:
demographic_file = os.path.join(site_folder,f'MELD_participants_infos_{site_code}.csv')
subjects = pd.read_csv(demographic_file)['id']  
# subjects = glob.glob(os.path.join(BIDS_path, '*'))
len(subjects) 

40

In [86]:
BIDS_path= os.path.join(site_folder, 'MELD_BIDS')

df=pd.DataFrame()

values={}
for subject in subjects :
    subject_bids = 'sub-'+harmonise_meld_name(subject)
    values['ID']=subject
    for mod in ['preop_T1w', 'postop_T1w','preop_T2w','preop_FLAIR','preop_DWI', 'lesion_MASK']:
        niis =  glob.glob(os.path.join(BIDS_path, subject_bids, f'*/*{mod}*.nii*'))
        jsons =  glob.glob(os.path.join(BIDS_path, subject_bids, f'*/*{mod}*.json'))
        
        values[f'{mod}_nii'] = len(niis)
        values[f'{mod}_json'] = len(jsons)

    df=pd.concat([df,pd.DataFrame([values])])

In [87]:
# overview of the MRI data available
df.sum()

ID                  MELD_H102_P_0001MELD_H102_P_0002MELD_H102_P_00...
preop_T1w_nii                                                      40
preop_T1w_json                                                     40
postop_T1w_nii                                                     37
postop_T1w_json                                                    37
preop_T2w_nii                                                      34
preop_T2w_json                                                     34
preop_FLAIR_nii                                                    40
preop_FLAIR_json                                                   40
preop_DWI_nii                                                       0
preop_DWI_json                                                      0
lesion_MASK_nii                                                    32
lesion_MASK_json                                                   32
dtype: object

In [67]:
# indicate who does not have a preop t1
df[(df.preop_DWI_nii<1) & (df.preop_DWI_json>0)]

,ID,preop_T1w_nii,preop_T1w_json,postop_T1w_nii,postop_T1w_json,preop_T2w_nii,preop_T2w_json,preop_FLAIR_nii,preop_FLAIR_json,preop_DWI_nii,preop_DWI_json,lesion_MASK_nii,lesion_MASK_json
0,MELD_H14_P_0023,1,1,0,1,0,1,1,1,0,1,1,1
0,MELD_H14_P_0025,0,1,0,0,0,1,0,1,0,1,1,1


In [68]:
df.loc['Column_Total'] = df.sum(numeric_only=True, axis=0)
df.loc[:,'Row_Total'] = df.sum(numeric_only=True, axis=1)

In [69]:
#save summary 
file = os.path.join(site_folder,'summary_MRI_MELDBIDS.csv')
df.to_csv(file)